In [3]:
#I need urllib.request to pull the info from the website
import urllib
import urllib.request

from bs4 import BeautifulSoup

#this creates the soup for the webpage
html_doc = urllib.request.urlopen('https://www.allrecipes.com/recipe/259356/roast-chicken-with-skillet-stuffing/')
soup = BeautifulSoup(html_doc, 'html.parser')

#All Ingredients have the class: ingredients-item-name
spans_ingredients = soup.find_all('span',{'class':'ingredients-item-name'})
ingredients = []
for span in spans_ingredients:
        ingredients.append(span.text)
# print(ingredients)

#All steps and recommendations have the class:paragraph
steps = []
find_steps = soup.find_all('div', {'class':'paragraph'})
for div in find_steps:
    steps.append(div.text)
for step in steps:
    print(step)
    print("")


Preheat oven to 375 degrees F (190 degrees C). 

Heat olive oil and 2 tablespoons butter in a large oven-safe skillet over medium heat. Add onions, celery, and leek. Cook and stir until softened, about 10 minutes. Stir in lemon zest, thyme, garlic, and red pepper flakes. Cook until flavors combine, about 5 minutes. Mix in parsley. Remove from heat; transfer vegetable mixture to a bowl. 

Butterfly the chicken by removing the backbone, and pressing down on the breastbone to crack chicken open using a knife or kitchen shears. Open the 2 sides and spread them out like an open book. Rub the remaining 2 tablespoons butter all over the chicken. Season with salt and pepper. 

Lay bread onto the bottom of the same skillet; spread vegetable mixture over bread to make stuffing. Arrange chicken into a layer on top, skin side-up. Pour lemon juice over chicken. 

Bake, uncovered, in the preheated oven until no longer pink at the bone and the juices run clear, 1 hour to 1 hour 30 minutes. An instant

In [17]:
import spacy
import rapidfuzz

nlp = spacy.load("en_core_web_sm")

cooking_utensils = ['oven', 'baking pan','baking sheet','barbecue grill','baster','basting brush','blender','bread basket','bread knife','Bundt pan','butcher block','cake pan','can opener','carafe','casserole pan','charcoal grill','cheese cloth','coffee maker','coffee pot','colander','convection oven','cookbook','cookie cutter','cookie press','cookie sheet','cooling rack','corer','crepe pan','crock','crock pot','cupcake pan','custard cup','cutlery','cutting board','Dutch oven','egg beater','egg poacher','egg timer','espresso machine','fondue pot','food processor','fork','frying pan','garlic press','gelatin mold','grater','griddle','grill pan','grinder','hamburger press','hand mixer','honey pot','ice bucket','ice cream scoop','icing spatula','infuser','jar opener','jellyroll pan','juicer','kettle','knife','ladle','lasagne pan','lid','mandolin','measuring cup','measuring spoon','microwave oven','mixing bowl','mold','mortar and pestle','muffin pan','nut cracker','oven','oven mitts','pan','parchment paper','paring knife','pastry bag','peeler','pepper mill','percolator','pie pan','pitcher','pizza cutter','pizza stone','platter','poacher','popcorn popper','pot','pot holder','poultry shears','pressure cooker','quiche pan','raclette grill','ramekin','refrigerator','rice cooker','ricer','roaster','roasting pan','rolling pin','salad bowl','salad spinner','salt shaker','sauce pan','scissors','sharpening steel','shears','sieve','skewer','skillet','slicer','slow cooker','souffle dish','spice rack','spoon','steak knife','steamer','stockpot','stove','strainer','tablespoon','tart pan','tea infuser','teakettle','teaspoon','thermometer','toaster','toaster oven','tongs','trivet','utensils','vegetable bin','vegetable peeler','waffle iron','water filter','whisk','wok','yogurt maker','zester']
cooking_mediums = ['olive oil']
cooking_actions = ['baking', 'frying', 'roasting', 'grilling', 'steaming', 'poaching', 'simmering', 'broiling', 'blanching', 'braising', 'stewing']

steps_dict = {}
step_counter = 1
substep_counter = 1
for step in steps:
    this_step = steps_dict[step_counter] = {}
    substeps = step.split('.')
    for substep in substeps:
        if len(substep.strip()) == 0:
            substep_counter += 1
            continue
        else:
            this_substep = this_step[substep_counter] = {'original_text': substep, 'tools': [], 'ingredients': [], 'conditions': []}
            text = nlp(substep)
            for token in text:
                if token.pos_ == 'VERB':
                    this_substep['action'] = token.text
                elif token.pos_ == 'ADJ':
                    this_substep['conditions'].append(token.text)
                elif token.pos_ == 'NUM':
                    this_substep['conditions'].append(token.text)
                elif token.pos_ == 'NOUN':
                    if token.text.lower() in cooking_utensils:
                        this_substep['tools'].append(token.text)
                    else:
                        found_match = False
                        for ingredient in ingredients:
                            if rapidfuzz.fuzz.partial_ratio(ingredient, token.text) > 80:
                                this_substep['ingredients'].append(token.text)
                                found_match = True
                                break
                        if not found_match:
                            this_substep['conditions'].append(token.text)
            substep_counter += 1
    step_counter += 1
    substep_counter = 1
    
for step in steps_dict:
    print(steps_dict[step])
    print("")
                
                
        
        
        
    

{1: {'original_text': 'Preheat oven to 375 degrees F (190 degrees C)', 'tools': [], 'ingredients': [], 'conditions': ['375', 'degrees', '190', 'degrees', 'C'], 'action': 'Preheat'}}

{1: {'original_text': 'Heat olive oil and 2 tablespoons butter in a large oven-safe skillet over medium heat', 'tools': ['skillet'], 'ingredients': ['olive', 'oil', 'tablespoons', 'butter'], 'conditions': ['Heat', '2', 'large', 'oven', 'safe', 'medium', 'heat']}, 2: {'original_text': ' Add onions, celery, and leek', 'tools': [], 'ingredients': ['onions', 'celery', 'leek'], 'conditions': [], 'action': 'Add'}, 3: {'original_text': ' Cook and stir until softened, about 10 minutes', 'tools': [], 'ingredients': [], 'conditions': ['10', 'minutes'], 'action': 'softened'}, 4: {'original_text': ' Stir in lemon zest, thyme, garlic, and red pepper flakes', 'tools': [], 'ingredients': ['lemon', 'thyme', 'garlic', 'pepper', 'flakes'], 'conditions': ['zest', 'red'], 'action': 'Stir'}, 5: {'original_text': ' Cook until f

In [19]:
import nltk

nlp = spacy.load("en_core_web_sm")

cooking_utensils = ['oven', 'baking pan','baking sheet','barbecue grill','baster','basting brush','blender','bread basket','bread knife','Bundt pan','butcher block','cake pan','can opener','carafe','casserole pan','charcoal grill','cheese cloth','coffee maker','coffee pot','colander','convection oven','cookbook','cookie cutter','cookie press','cookie sheet','cooling rack','corer','crepe pan','crock','crock pot','cupcake pan','custard cup','cutlery','cutting board','Dutch oven','egg beater','egg poacher','egg timer','espresso machine','fondue pot','food processor','fork','frying pan','garlic press','gelatin mold','grater','griddle','grill pan','grinder','hamburger press','hand mixer','honey pot','ice bucket','ice cream scoop','icing spatula','infuser','jar opener','jellyroll pan','juicer','kettle','knife','ladle','lasagne pan','lid','mandolin','measuring cup','measuring spoon','microwave oven','mixing bowl','mold','mortar and pestle','muffin pan','nut cracker','oven','oven mitts','pan','parchment paper','paring knife','pastry bag','peeler','pepper mill','percolator','pie pan','pitcher','pizza cutter','pizza stone','platter','poacher','popcorn popper','pot','pot holder','poultry shears','pressure cooker','quiche pan','raclette grill','ramekin','refrigerator','rice cooker','ricer','roaster','roasting pan','rolling pin','salad bowl','salad spinner','salt shaker','sauce pan','scissors','sharpening steel','shears','sieve','skewer','skillet','slicer','slow cooker','souffle dish','spice rack','spoon','steak knife','steamer','stockpot','stove','strainer','tablespoon','tart pan','tea infuser','teakettle','teaspoon','thermometer','toaster','toaster oven','tongs','trivet','utensils','vegetable bin','vegetable peeler','waffle iron','water filter','whisk','wok','yogurt maker','zester']
cooking_mediums = ['olive oil']
cooking_actions = ['baking', 'frying', 'roasting', 'grilling', 'steaming', 'poaching', 'simmering', 'broiling', 'blanching', 'braising', 'stewing']

steps_dict = {}
step_counter = 1
substep_counter = 1
for step in steps:
    this_step = steps_dict[step_counter] = {}
    substeps = step.split('.')
    for substep in substeps:
        if len(substep.strip()) == 0:
            substep_counter += 1
            continue
        else:
            this_substep = this_step[substep_counter] = {'original_text': substep, 'tools': [], 'ingredients': [], 'conditions': []}
            text = nlp(substep)
            for token in text:
                if token.pos_ == 'VERB':
                    this_substep['action'] = token.text
                elif token.pos_ == 'ADJ':
                    this_substep['conditions'].append(token.text)
                elif token.pos_ == 'NUM':
                    this_substep['conditions'].append(token.text)
                elif token.pos_ == 'NOUN':
                    if token.text.lower() in cooking_utensils:
                        this_substep['tools'].append(token.text)
                    else:
                        found_match = False
                        for ingredient in ingredients:
                            if rapidfuzz.fuzz.partial_ratio(ingredient, token.text) > 80:
                                this_substep['ingredients'].append(token.text)
                                found_match = True
                                break
                        if not found_match:
                            this_substep['conditions'].append(token.text)
            substep_counter += 1
    step_counter += 1
    substep_counter = 1
    
for step in steps_dict:
    print(steps_dict[step])
    print("")
                
                
        
        
        
    

Heat
NOUN
olive
NOUN
oil
NOUN
and
CCONJ
2
NUM
tablespoons
NOUN
butter
NOUN
in
ADP
a
DET
large
ADJ
oven
ADJ
-
PUNCT
safe
ADJ
skillet
NOUN
over
ADP
medium
ADJ
heat
NOUN
